In [1]:
import os
import apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText

In [3]:
# DoFn to perform on each element in the input PCollection.
class ActorCountFn(beam.DoFn):
  def process(self, element):
    record = element
    year = record.get('year')
    category = record.get('category')
    winner = record.get('winner')
    entity = record.get('entity')

    if 'ACTOR' in category or 'ACTRESS' in category:
        return [(entity, 1)]    

# DoFn to perform on each element in the input PCollection.
class ActorSumFn(beam.DoFn):
  def process(self, element):
     actor, count_obj = element # count_obj is an _UnwindowedValues type
     count_list = list(count_obj) # cast to list to support len
     total_count = len(count_list)
     return [(actor, total_count)]  
    

PROJECT_ID = '<YOUR PROJECT ID>'

# Project ID is needed for BigQuery data source, even for local execution.
options = {
    'project': PROJECT_ID
}
opts = beam.pipeline.PipelineOptions(flags=[], **options)

# Create a Pipeline using a local runner for execution.
with beam.Pipeline('DirectRunner', options=opts) as p:

    query_string = 'SELECT * FROM Nomination_Events'
    query_results = p | 'Read from BigQuery' >> beam.io.Read(beam.io.BigQuerySource(query=query_string))

    # write PCollection to log file
    query_results | 'Write to log 1' >> WriteToText('query_results.txt')

    # apply a ParDo to the PCollection 
    actor_pcoll = query_results | 'Extract Actor' >> beam.ParDo(ActorCountFn())

    # write PCollection to log file
    actor_pcoll | 'Write to log 2' >> WriteToText('actor_count.txt')

    # apply GroupByKey to the PCollection
    group_pcoll = actor_pcoll | 'Group by Actor' >> beam.GroupByKey()

    # write PCollection to log file
    group_pcoll | 'Write to log 3' >> WriteToText('group_by_actor.txt')
  
    # apply a ParDo to the PCollection
    out_pcoll = group_pcoll | 'Sum up Counts' >> beam.ParDo(ActorSumFn())

    # write PCollection to a file
    out_pcoll | 'Write File' >> WriteToText('oscars_output.txt')


ERROR:root:Exception at bundle <apache_beam.runners.direct.bundle_factory._Bundle object at 0x7f4a37f1e3b0>, due to an exception.
 Traceback (most recent call last):
  File "/home/jupyter/beam_venv_dir/local/lib/python2.7/site-packages/apache_beam/runners/direct/executor.py", line 341, in call
    finish_state)
  File "/home/jupyter/beam_venv_dir/local/lib/python2.7/site-packages/apache_beam/runners/direct/executor.py", line 381, in attempt_call
    result = evaluator.finish_bundle()
  File "/home/jupyter/beam_venv_dir/local/lib/python2.7/site-packages/apache_beam/runners/direct/transform_evaluator.py", line 306, in finish_bundle
    bundles = _read_values_to_bundles(reader)
  File "/home/jupyter/beam_venv_dir/local/lib/python2.7/site-packages/apache_beam/runners/direct/transform_evaluator.py", line 293, in _read_values_to_bundles
    read_result = [GlobalWindows.windowed_value(e) for e in reader]
  File "/home/jupyter/beam_venv_dir/local/lib/python2.7/site-packages/apache_beam/io/gcp/

HttpBadRequestError: HttpError accessing <https://www.googleapis.com/bigquery/v2/projects/cs327e-fa2019/queries/38f96d7355814675be45b90731273f9a?alt=json&maxResults=10000>: response: <{'status': '400', 'content-length': '362', 'x-xss-protection': '0', 'x-content-type-options': 'nosniff', 'transfer-encoding': 'chunked', 'vary': 'Origin, X-Origin, Referer', 'server': 'ESF', '-content-encoding': 'gzip', 'cache-control': 'private', 'date': 'Mon, 18 Nov 2019 22:13:44 GMT', 'x-frame-options': 'SAMEORIGIN', 'content-type': 'application/json; charset=UTF-8'}>, content <{
  "error": {
    "code": 400,
    "message": "Table name \"Nomination_Events\" cannot be resolved: dataset name is missing.",
    "errors": [
      {
        "message": "Table name \"Nomination_Events\" cannot be resolved: dataset name is missing.",
        "domain": "global",
        "reason": "invalid"
      }
    ],
    "status": "INVALID_ARGUMENT"
  }
}
>